In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
columns = ['id','country','Label','Text']
df = pd.read_csv("twitter_training.csv", names=columns)

In [3]:
df = df.head(5000)

In [4]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [5]:
model_name = f"gunkaynar/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [6]:
example = "I hate my job"

In [7]:
#Roberta Model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

{'roberta_neg': 0.9655635, 'roberta_neu': 0.034436498, 'roberta_pos': 3.514553e-08}


In [8]:
def scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [9]:
token_val = [str(i) for i in df['Text'].values]

In [12]:
res = {}
for i in range(len(token_val)):
    try:
        text = token_val[i]
        res[i] = scores_roberta(text)
    except RuntimeError:
        print(f'Broke for id {i}')

In [10]:
len(token_val)

5000

In [13]:
len(res)

5000

In [15]:
results_df = pd.DataFrame(res).T

In [16]:
df.join(results_df)

,id,country,Label,Text,roberta_neg,roberta_neu,roberta_pos
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,0.810235,0.189765,1.163415e-07
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,0.803057,0.196943,8.474538e-08
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,0.731596,0.268403,2.948823e-07
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,0.823887,0.176113,1.211009e-07
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,0.690506,0.309493,6.347284e-07
...,...,...,...,...,...,...,...
4995,58,Amazon,Neutral,@OregonChai did finally stop making the sugar ...,0.690118,0.309882,2.762316e-07
4996,58,Amazon,Neutral,@OregonChai What did y ’ all really stop makin...,0.580994,0.419004,1.784508e-06
4997,58,Amazon,Neutral,@OregonChai did y’all stop making the butter<u...,0.584411,0.415588,1.062763e-06
4998,59,Amazon,Neutral,Amazon UK launches the Sherlock Holmes Advent ...,0.865597,0.134403,4.594827e-08
